ETL_australian_user_items.json


Importo las librerías 

In [1]:
import json
import ast
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
row = []  # creamos una lista vacia para ir agregando las filas del archivo json

with open("Data/users_items.json", encoding="utf-8") as file:
    for line in file:
        # Utilizamos ast.literal_eval para interpretar la línea como expresión literal de Python
        # Luego, convertimos la expresión evaluada a formato JSON utilizando json.dumps
        row.append(json.loads(json.dumps(ast.literal_eval(line))))

# Creamos un dataframe con la lista de diccionarios
items = pd.DataFrame(row)
items.head()


,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [3]:
items.shape

(88310, 5)

TRANSFORMACIÓN DE DATASET
El proceso de transformación del dataset implica la desanidación de una lista con diccionarios contenida en la columna "items". La columna "items" presenta datos anidados, constituyendo una lista de diccionarios. Para abordar esta estructura, se emplea un método de normalización que conserva la columna "items" como el eje organizador, permitiendo mantener las claves de los diccionarios como columnas individuales en el dataset resultante.

In [3]:
# Normalizamos la columna 'items'
normalizado = pd.json_normalize(row, record_path=['items'], meta=['steam_id','items_count','user_id', 'user_url'] )
normalizado.head()

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


In [5]:
normalizado.shape

(5153209, 8)

In [4]:
tipo_data = {"columna":[],"tipos_de_datos":[]} # Generamos un diccionario vacio para ir almacenando lo que genere el bucle

for columna in normalizado.columns: # Recorremos las columnas del dataframe 
    tipo_data["columna"].append(columna) # Agregamos el nombre de la columna
    tipo_data["tipos_de_datos"].append(normalizado[columna].apply(type).unique()) # Agregamos los tipos de datos que hay en la columna

analisis= pd.DataFrame(tipo_data) # Creamos un dataframe con el diccionario
analisis

,columna,tipos_de_datos
0,item_id,[<class 'str'>]
1,item_name,[<class 'str'>]
2,playtime_forever,[<class 'int'>]
3,playtime_2weeks,[<class 'int'>]
4,steam_id,[<class 'str'>]
5,items_count,[<class 'int'>]
6,user_id,[<class 'str'>]
7,user_url,[<class 'str'>]


In [5]:
normalizado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           object
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   steam_id          object
 5   items_count       object
 6   user_id           object
 7   user_url          object
dtypes: int64(2), object(6)
memory usage: 314.5+ MB


BUSQUEDA DE DUPLICADOS 

En este dataframe la busqueda la búsqueda y eliminación nulos se realiza después de la normalización, debido a la gran cantidad de datos anidados en la columna "items", a modo de perder los datos sin ser analizados 

In [6]:
duplicados = normalizado.loc[normalizado.duplicated()] # Se utilza la variable duplicados para guardar la busqueda y después compararlos
duplicados

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
164294,20,Team Fortress Classic,5,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164295,50,Half-Life: Opposing Force,0,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164296,70,Half-Life,0,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164297,130,Half-Life: Blue Shift,0,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164298,220,Half-Life 2,198,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
...,...,...,...,...,...,...,...,...
4898223,213670,South Park™: The Stick of Truth™,725,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898224,221910,The Stanley Parable,53,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898225,261030,The Walking Dead: Season Two,253,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898226,273110,Counter-Strike Nexon: Zombies,0,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...


SE ANALIZA LAS FILAS CON DATOS DUPLICADOS Y SE ELIMINAN 

In [7]:
normalizado = normalizado.drop_duplicates(keep='first')
normalizado

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153205,373330,All Is Dust,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


Observando que me encuentro con un Dataset con 5.094.105 de registros, genera una demora de carga, memoria del sistema.
No genera una optimización de los datos, se plantea revisar la cantidad de Filas donde las horas de juego sean 0 , donde ee encuentre una cantidad de registros importantes, se procede a eliminar.  


In [8]:
normalizado['playtime_forever'].value_counts()

0        1847730
1         101586
2          34391
3          31530
4          29127
          ...   
76541          1
64676          1
53328          1
44029          1
34753          1
Name: playtime_forever, Length: 48861, dtype: int64

In [9]:
# Filtrar y mantener solo las filas donde playtime_forever no es igual a 0
normalizado = normalizado[normalizado['playtime_forever'] != 0]

# Restablecer los índices después de filtrar
normalizado = normalizado.reset_index(drop=True)

In [10]:
normalizado['playtime_forever'].value_counts()

1        101586
2         34391
3         31530
4         29127
5         27612
          ...  
76541         1
64676         1
53328         1
44029         1
34753         1
Name: playtime_forever, Length: 48860, dtype: int64

La columna "playtime_forever" se encuentra en minutos, decido a cambiarla por horas, por lo tanto creo una 
nueva columna. 

In [11]:
# Creamos la columna 'playtime_forever_hours' que es el resultado de dividir 'playtime_forever' por 60 y redondeando a 2 decimales
normalizado['playtime_hours'] = round(normalizado['playtime_forever']/60,2)

Se visualiza la cantidad de horas jugadas. 

In [12]:
normalizado['playtime_hours'].value_counts() # vemos la cantidad de horas jugadas

0.02       101586
0.03        34391
0.05        31530
0.07        29127
0.08        27612
            ...  
1275.68         1
1077.93         1
888.80          1
733.82          1
579.22          1
Name: playtime_hours, Length: 48860, dtype: int64

Observando que la columna "playtime_forever y "playtime_2weeks" las encuentro inecesarias para el análisis, procedo a eliminarlas de esta manera reduzco los datos del dataset.	
	

In [13]:
# Eliminamos las columnas 'playtime_2weeks' y 'playtime_2weeks' por no ser de utilidad para el análisis
normalizado = normalizado.drop(['playtime_2weeks','playtime_forever'], axis=1)

In [14]:
normalizado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3246375 entries, 0 to 3246374
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   item_id         object 
 1   item_name       object 
 2   steam_id        object 
 3   items_count     object 
 4   user_id         object 
 5   user_url        object 
 6   playtime_hours  float64
dtypes: float64(1), object(6)
memory usage: 173.4+ MB


La columna "user_url" la encuentro no necesaria, dado que la información que brinda es solo la url del usuario 
en la plataforma de Steam. 

In [15]:
# Eliminamos la columna 'usr_url'
normalizado = normalizado.drop(['user_url'], axis=1)

Guardo el Datafarme para las consultas, transformandolo en una copia para no modificar el original, luego pasamos a guardarlo en una archivo.csv y por ultimo lo guardamos en una archivo parquet para optimizar su utilización. 

In [17]:
normalizado = normalizado.copy() # Copiamos el dataframe para no modificar el original

In [18]:
normalizado.to_csv("Data/users_items.csv", index=False, encoding="utf-8") # Guardamos el dataframe en un archivo csv

Este código me genera un error que es el siguiente: "MemoryError" indica que no hay suficiente memoria disponible para completar la operación. Este error surge cuando se intenta cargar en la memoria RAM una cantidad de datos que excede la capacidad disponible.

In [19]:
normalizado = pd.read_csv("Data/users_items.csv") # Leemos el archivo csv

tabla = pa.Table.from_pandas(normalizado) # Convertimos el dataframe en una tabla de pyarrow
pq.write_table(tabla,"Data/users_items.parquet") # Guardamos la tabla en un archivo parquet

Investigo cual es la solución: Me encuentro que se puede procesar los datos por partes para reducir el uso de memoria. 
Se dividen los archivos CSV en segemntos pequeños y se guarda en archivos parquet por separado, y se logar el resultado. 



In [20]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [21]:
chunk_size = 100000  # Tamaño de los segmentos o chunks

In [22]:
# Iterar sobre el archivo CSV en bloques y convertir cada bloque en archivos Parquet
csv_file = "Data/users_items.csv"
parquet_file = "Data/users_items_{}.parquet"

In [23]:
reader = pd.read_csv(csv_file, chunksize=chunk_size)
i = 0

for chunk in reader:
    table = pa.Table.from_pandas(chunk)
    pq.write_table(table, parquet_file.format(i))
    i += 1